## Look at ASCOT data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from mtsplice import get_data_dir
from mtsplice.data.ascot_dl import tissues
import matplotlib.pyplot as plt
%matplotlib inline 

Using TensorFlow backend.
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint1

In [4]:
DATA = get_data_dir()

In [5]:
gtex = pd.read_csv(DATA+"gtex_psi.csv.gz")

In [6]:
pd.set_option('display.max_columns', None)  


In [7]:
gtex.head()

,exon_id,cassette_exon,alternative_splice_site_group,linked_exons,mutually_exclusive_exons,exon_strand,exon_length,gene_type,gene_id,gene_symbol,exon_location,exon_boundary,Retina - Eye,RPE/Choroid/Sclera - Eye,Subcutaneous - Adipose,Visceral (Omentum) - Adipose,Adrenal Gland,Aorta - Artery,Coronary - Artery,Tibial - Artery,Bladder,Amygdala - Brain,Anterior cingulate - Brain,Caudate nucleus - Brain,Cerebellar Hemisphere - Brain,Cerebellum - Brain,Cortex - Brain,Frontal Cortex - Brain,Hippocampus - Brain,Hypothalamus - Brain,Nucleus accumbens - Brain,Putamen - Brain,Spinal cord (C1) - Brain,Substantia nigra - Brain,Mammary Tissue - Breast,EBV-xform lymphocytes - Cells,Leukemia (CML) - Cells,Xform. fibroblasts - Cells,Ectocervix - Cervix,Endocervix - Cervix,Sigmoid - Colon,Transverse - Colon,Gastroesoph. Junc. - Esophagus,Mucosa - Esophagus,Muscularis - Esophagus,Fallopian Tube,Atrial Appendage - Heart,Left Ventricle - Heart,Cortex - Kidney,Liver,Lung,Minor Salivary Gland,Skeletal - Muscle,Tibial - Nerve,Ovary,Pancreas,Pituitary,Prostate,Not Sun Exposed - Skin,Sun Exposed (Lower leg) - Skin,Ileum - Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,GT_00001,Yes,No,No,No,-,87,protein_coding,ENSG00000121410.11,A1BG,chr19:58352098-58352184,chr19:58351688-58352282,-1.000000,0.000000,5.650253,12.284321,10.668563,8.910675,8.306491,7.042732,-1.0,3.389831,0.000000,4.845266,2.339823,2.992987,4.194915,2.572108,4.126146,4.916002,3.103517,0.000000,3.571429,0.000000,8.422531,8.522727,7.499975,9.270774,-1.000000,-1.000000,8.798250,11.041114,5.802975,7.240251,13.155539,15.213102,11.200752,9.005443,7.393484,9.152529,9.617292,5.785714,6.132789,7.575751,8.172850,16.716418,10.848389,12.887373,13.199024,12.135550,7.189542,12.656651,10.497812,8.512642,18.084153,8.528845,7.439244,9.999865
1,GT_00002,Yes,No,No,No,+,104,antisense,ENSG00000268895.5,A1BG-AS1,chr19:58352941-58353044,chr19:58347845-58353378,15.663411,16.781935,31.369611,29.965765,32.804476,33.395175,37.043674,31.704029,-1.0,13.086926,12.374233,12.375266,17.323693,15.989524,12.337049,10.450090,15.697992,13.571478,9.651863,15.295671,16.531789,14.127646,29.834951,36.878121,31.381492,21.006140,40.126961,23.085491,28.507092,37.186194,30.163296,32.535002,29.372090,36.904786,20.365901,32.073303,33.571429,37.154500,42.159326,26.853081,24.595114,22.691265,34.150483,33.987435,17.216046,37.302914,33.590115,30.011723,37.515735,36.528732,33.971923,27.981121,33.010940,34.812014,31.081544,40.580524
2,GT_00003,No,3',Yes,No,-,182|206,protein_coding,ENSG00000148584.14,A1CF,chr10:50813857-50814038|50814062,chr10:50811177-50816005,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.669993,99.801520,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.534133,99.410914,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.538740,-1.000000,-1.000000,-1.000000,-1.000000,99.803153,100.000000,100.000000,-1.000000,-1.000000,-1.000000,-1.000000,100.000000
3,GT_00004,No,3',Yes,No,-,2423,protein_coding,ENSG00000148584.14,A1CF,chr10:50813857-50816279,chr10:50811177-50820551,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.812041,99.728649,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.738215,98.937103,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,99.003317,-1.000000,-1.000000,-1.000000,-1.000000,99.791151,100.000000,100.000000,-1.000000,-1.000000,-1.000000,-1.000000,100.000000
4,GT_00005,Yes,No,No,No,-,165,protein_coding,ENSG00000148584.14,A1CF,chr10:50828131-50828295,chr10:50820650-50836073,-1.000000,-1.000000,-1.000000,-

In [8]:
gtex.shape

(80017, 68)

In [9]:
gtex.cassette_exon.describe()

count     80017
unique        2
top         Yes
freq      61823
Name: cassette_exon, dtype: object

1. How to deal with variable input length
    * Add intron to make sequence all same length
    * Scan sequence, take maximum donor acceptor score (first detection)

In [10]:
cassette = gtex[gtex.cassette_exon=='Yes']

In [11]:
cassette.shape

(61823, 68)

In [12]:
def exon_location2_coordinates(location):
    assert "|" not in location
    chrom, coordinate = location.strip("\t").split(':')
    start, end = coordinate.split('-')
    return (chrom, int(start), int(end))

In [13]:
coordinate = pd.DataFrame(cassette['exon_location'].apply(exon_location2_coordinates).tolist(), columns=['chrom', 'exon_start', 'exon_end'])

In [14]:
intron = pd.DataFrame(cassette['exon_boundary'].apply(exon_location2_coordinates).tolist(), columns=['chrom', 'intron_start', 'intron_end'])

In [15]:
intron = intron[['intron_start', 'intron_end']]

In [16]:
coordinate.head()

,chrom,exon_start,exon_end
0,chr19,58352098,58352184
1,chr19,58352941,58353044
2,chr10,50828131,50828295
3,chr10,50833145,50833180
4,chr10,50841862,50841992


In [17]:
cassette = pd.concat([coordinate, intron, cassette.reset_index()], axis=1)

In [18]:
cassette.head()

,chrom,exon_start,exon_end,intron_start,intron_end,index,exon_id,cassette_exon,alternative_splice_site_group,linked_exons,mutually_exclusive_exons,exon_strand,exon_length,gene_type,gene_id,gene_symbol,exon_location,exon_boundary,Retina - Eye,RPE/Choroid/Sclera - Eye,Subcutaneous - Adipose,Visceral (Omentum) - Adipose,Adrenal Gland,Aorta - Artery,Coronary - Artery,Tibial - Artery,Bladder,Amygdala - Brain,Anterior cingulate - Brain,Caudate nucleus - Brain,Cerebellar Hemisphere - Brain,Cerebellum - Brain,Cortex - Brain,Frontal Cortex - Brain,Hippocampus - Brain,Hypothalamus - Brain,Nucleus accumbens - Brain,Putamen - Brain,Spinal cord (C1) - Brain,Substantia nigra - Brain,Mammary Tissue - Breast,EBV-xform lymphocytes - Cells,Leukemia (CML) - Cells,Xform. fibroblasts - Cells,Ectocervix - Cervix,Endocervix - Cervix,Sigmoid - Colon,Transverse - Colon,Gastroesoph. Junc. - Esophagus,Mucosa - Esophagus,Muscularis - Esophagus,Fallopian Tube,Atrial Appendage - Heart,Left Ventricle - Heart,Cortex - Kidney,Liver,Lung,Minor Salivary Gland,Skeletal - Muscle,Tibial - Nerve,Ovary,Pancreas,Pituitary,Prostate,Not Sun Exposed - Skin,Sun Exposed (Lower leg) - Skin,Ileum - Small Intestine,Spleen,Stomach,Testis,Thyroid,Uterus,Vagina,Whole Blood
0,chr19,58352098,58352184,58351688,58352282,0,GT_00001,Yes,No,No,No,-,87,protein_coding,ENSG00000121410.11,A1BG,chr19:58352098-58352184,chr19:58351688-58352282,-1.000000,0.000000,5.650253,12.284321,10.668563,8.910675,8.306491,7.042732,-1.0,3.389831,0.000000,4.845266,2.339823,2.992987,4.194915,2.572108,4.126146,4.916002,3.103517,0.000000,3.571429,0.000000,8.422531,8.522727,7.499975,9.270774,-1.000000,-1.000000,8.798250,11.041114,5.802975,7.240251,13.155539,15.213102,11.200752,9.005443,7.393484,9.152529,9.617292,5.785714,6.132789,7.575751,8.172850,16.716418,10.848389,12.887373,13.199024,12.135550,7.189542,12.656651,10.497812,8.512642,18.084153,8.528845,7.439244,9.999865
1,chr19,58352941,58353044,58347845,58353378,1,GT_00002,Yes,No,No,No,+,104,antisense,ENSG00000268895.5,A1BG-AS1,chr19:58352941-58353044,chr19:58347845-58353378,15.663411,16.781935,31.369611,29.965765,32.804476,33.395175,37.043674,31.704029,-1.0,13.086926,12.374233,12.375266,17.323693,15.989524,12.337049,10.450090,15.697992,13.571478,9.651863,15.295671,16.531789,14.127646,29.834951,36.878121,31.381492,21.006140,40.126961,23.085491,28.507092,37.186194,30.163296,32.535002,29.372090,36.904786,20.365901,32.073303,33.571429,37.154500,42.159326,26.853081,24.595114,22.691265,34.150483,33.987435,17.216046,37.302914,33.590115,30.011723,37.515735,36.528732,33.971923,27.981121,33.010940,34.812014,31.081544,40.580524
2,chr10,50828131,50828295,50820650,50836073,4,GT_00005,Yes,No,No,No,-,165,protein_coding,ENSG00000148584.14,A1CF,chr10:50828131-50828295,chr10:50820650-50836073,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,95.756119,94.935577,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,98.266500,96.200084,100.000000,-1.000000,-1.000000,-1.000000,-1.000000,96.491756,-1.000000,-1.000000,-1.000000,-1.000000,93.836895,96.076955,98.374384,-1.000000,-1.000000,-1.000000,-1.000000,98.235610
3,chr10,50833145,50833180,50828296,50836073,5,GT_00006,Yes,No,No,No,-,36,protein_coding,ENSG00000148584.14,A1CF,chr10:50833145-50833180,chr10:50828296-50836073,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,-1.0,0.000000,-1.000000,-1.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,2.096629,1.874037,0.000000,-1.000000,0.000000,-1.000000,0.000000,0.000000,0.271736,0.655501,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.510681,1.922885,0.000000,0.000000,0.000000,0.00

In [19]:
cassette[['intron_start', 'exon_start', 'exon_end', 'intron_end', 'exon_strand']].head()

,intron_start,exon_start,exon_end,intron_end,exon_strand
0,58351688,58352098,58352184,58352282,-
1,58347845,58352941,58353044,58353378,+
2,50820650,50828131,50828295,50836073,-
3,50828296,50833145,50833180,50836073,-
4,50836313,50841862,50841992,50843987,-


In [20]:
cassette = cassette.drop('index', axis=1)

In [21]:
cassette = cassette.rename(columns={"exon_strand": "strand"})

### Apply some filtering

In [22]:
X = cassette[tissues].values

In [23]:
X[X==-1] = np.nan
X = X / 100.

In [24]:
from kipoi.writers import HDF5BatchWriter
from mtsplice import get_paper_dir

In [25]:
all_cassette = cassette[['chrom', 'exon_start', 'exon_end', 'strand']]
all_cassette['ref_psi_mean'] = np.nanmean(X, axis=-1)

/opt/modules/i12g/anaconda/3-5.0.1/envs/mtsplice_paper/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [26]:
all_cassette.to_csv(DATA + "ascot_psi_mean.csv.gz", compression='gzip')

## Split into train validation and test

In [27]:
TRAIN_CHROMS = ['4', '6', '8'] + list(range(10,23)) + ['X', 'Y']
TRAIN_CHROMS = ['chr'+str(i) for i in TRAIN_CHROMS]
TEST_CHROMS = ['chr2', 'chr3', 'chr5']
VAL_CHROMS = ['chr1','chr7','chr9']

In [28]:
train = cassette[cassette.chrom.isin(TRAIN_CHROMS)].reset_index(drop=True)
test = cassette[cassette.chrom.isin(TEST_CHROMS)].reset_index(drop=True)
val = cassette[cassette.chrom.isin(VAL_CHROMS)].reset_index(drop=True)